# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847727350622                   -0.70    4.5   28.7ms
  2   -7.852399974528       -2.33       -1.53    1.0   17.0ms
  3   -7.852609612559       -3.68       -2.54    1.2   17.4ms
  4   -7.852645445696       -4.45       -2.77    2.5   48.0ms
  5   -7.852646084943       -6.19       -2.87    1.0   17.5ms
  6   -7.852646665599       -6.24       -3.83    1.0   17.7ms
  7   -7.852646684886       -7.71       -4.66    1.5   19.4ms
  8   -7.852646686679       -8.75       -5.01    1.8   20.6ms
  9   -7.852646686705      -10.58       -5.13    1.2   18.6ms
 10   -7.852646686727      -10.66       -5.58    1.0   17.9ms
 11   -7.852646686730      -11.54       -6.16    1.2   18.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847752665561                   -0.70           4.5    401ms
  2   -7.852562076064       -2.32       -1.62   0.80    2.2    2.30s
  3   -7.852641328256       -4.10       -2.70   0.80    1.0    227ms
  4   -7.852646511100       -5.29       -3.40   0.80    1.8   18.8ms
  5   -7.852646681980       -6.77       -4.43   0.80    2.0   18.8ms
  6   -7.852646686627       -8.33       -4.91   0.80    2.2   20.7ms
  7   -7.852646686726      -10.01       -5.55   0.80    1.2   16.4ms
  8   -7.852646686730      -11.45       -6.72   0.80    1.8   17.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.348191839589                   -1.05    4.61s
  2   -1.431696375520        0.44       -0.65    105ms
  3   -3.656713203952        0.35       -0.43   32.7ms
  4   -4.847730875617        0.08       -0.56   32.7ms
  5   -6.572732137641        0.24       -0.70   32.7ms
  6   -7.335425628804       -0.12       -1.06   32.7ms
  7   -7.608493007253       -0.56       -1.44   24.6ms
  8   -7.714953683312       -0.97       -1.92   24.7ms
  9   -7.743786113871       -1.54       -1.84   24.6ms
 10   -7.775443711426       -1.50       -2.16   24.7ms
 11   -7.798934401337       -1.63       -2.14   24.7ms
 12   -7.817784447370       -1.72       -2.14   24.7ms
 13   -7.835645758700       -1.75       -2.24   24.8ms
 14   -7.847399932752       -1.93       -2.27   24.7ms
 15   -7.850262882215       -2.54       -3.12   25.6ms
 16   -7.851992700991       -2.76       -2.88   24.7ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847728572261                   -0.70    4.8   34.8ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645821338                   -1.63    13.6s
  2   -7.852646686730       -6.06       -3.69    3.60s
  3   -7.852646686730      -13.16       -7.18    126ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.786126011006329e-7
|ρ_newton - ρ_scfv| = 4.813174774539961e-7
|ρ_newton - ρ_dm|   = 1.1456275422720287e-9
